# Tutorial: Functional annotation analysis with MOSHPIT

This notebook contains materials accompanying the Rigi Workshop 2025: **Microbiome Meets Metabolism**. The notebook and corresponding setup script were adapted from the [**Advanced Block Course: Computational Biology**](https://github.com/bokulich-lab/advanced-comp-bio-tutorial.git); all source code is licensed under the Apache License 2.0.

Save your own local copy of this notebook by using `File > Save a copy in Drive`. At some point you may be prompted to trust the notebook. We promise that it is safe 🤞

### Background

In this tutorial, we will analyze functional gene annotation data derived from [metagenome](https://en.wikipedia.org/wiki/Metagenomics) sequence data using [MOSHPIT](https://moshpit.readthedocs.io/en/latest/intro.html), a a toolkit of plugins for whole metagenome assembly, annotation, and analysis built on the microbiome multi-omics data science framework [QIIME 2](https://qiime2.org/).

This tutorial is based on one chapter of the [MOSHPIT tutorial](https://moshpit.readthedocs.io/en/latest/intro.html). Refer to this tutorial for complete instructions for running MOSHPIT, including steps for functional annotation of metagenome data used to generate the dataset used below.

Given the timeframe of this workshop and the high level of computational resources and time required for typical metagenome analysis, we will start with pre-computed functional gene annotation data and learn some options for analyzing and visualizing these data downstream.

In this notebook, we will re-analyze data from [this study](https://www.sciencedirect.com/science/article/pii/S0740002020301970) to investigate differences in functional gene profiles between cocoa bean fermentations of two different cultivars (Forasteiro and Hybrid) at two different stages of fermentation (early and late)

### Environment setup

MOSHPIT is usually installed by following the [official installation instructions](https://docs.qiime2.org/2024.10/install/). However, because we are using Google Colab and there are some caveats to using conda here, we will have to hack around the installation a little. But no worries, we provide a setup script below which does all this work for us. 😌 Let's start by pulling a local copy of the project repository down from GitHub.

From here, you run the entire notebook by selecting `Runtime > Run all` from the menu in Google Colab. Some steps are time-comsuming and the entire notebook may take up to 30-60 minutes, so run the entire notebook now and we will inspect the commands and results as we work through as a class.

🛑 **ACTION** 🛑
<br>
*Run every cell in the notebook using the instructions above.*

Note that the Environment Setup section should be skipped if you are trying to run this notebook outside of Google Colab. After cloning the repository we will change the working directory to `materials` and execute the setup script - this should take around 10 minutes. Finally, we will alias the "mosh" command to point to the moshpit-dev environment.

In [ ]:
! git clone https://github.com/fsb-edu/rigi-workshop.git materials
%cd materials
%run setup_moshpit
%alias mosh mamba run -n moshpit-dev -r /usr/local mosh

# 1.0 Functional annotation with MOSHPIT

## 1.1. Extracting Annotation Data

The data that we downloaded above includes functional annotation with multiple different databases/ontologies. As the study subject here is cocoa fermentation, we will first narrow down the list of proteins of interest to enzymes involved in carbon metabolism (CAZymes). To do that we can use the `filter-annotations` action [exclusive preview!]:

In [ ]:
mosh annotate filter-annotations --i-ortholog-annotations ./data/eggnog_annotations.qza --p-query "CAZy<>'-'" --o-filtered-annotations ./data/eggnog_annotations_filtered.qza

Next, we will use the following action to extract a specific annotation from the filtered table generated by EggNOG and calculate its frequencies across all MAGs.

The table resulting from this action will provide information about the frequency of each KEGG pathway observed in each MAG:

In [ ]:
mosh annotate extract-annotations --i-ortholog-annotations ./data/eggnog_annotations_filtered.qza --p-annotation kegg_reaction --p-max-evalue 0.0001 --o-annotation-frequency ./data/eggnog_kegg_reaction_freq.qza

## 1.2. Calculating abundance of pathways per sample

What if we want to count KEGG or other pathway information per sample? We derive this information via [matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication) of pathway count per MAG X the count of each MAG per sample. This can be useful for aggregating pathway information across many MAGs per sample, e.g., to identify pathways that are enriched in individual samples or groups of samples.

The action below will perform matrix multiplication of the pathway-per-MAG feature table X MAG-per-sample feature table to generate a pathway-per-sample feature table as depicted in the diagram above.


In [ ]:
mosh annotate multiply-tables --i-table1 ./data/mags_derep_ft.qza --i-table2 ./data/eggnog_kegg_reaction_freq.qza --o-result-table ./data/kegg_reaction_ft.qza

## 1.3. Pathway Enrichment

PCoA is a useful technique for visualizing similarity between samples based on composition data, but does not tell us whether thes differences are significant, or which features (e.g., pathways) are enriched in specific groups. For this, we can use various statistical techniques to test which features are associated with specific sample metadata information.

First, we will use the [ANCOM-BC](https://www.nature.com/articles/s41467-020-17041-7) test to identify pathways that are enriched during the late stages of fermentation compared to the early ones, and visualize these as a barplot.

In [ ]:
mosh composition ancombc --i-table ./data/kegg_reaction_ft.qza --m-metadata-file ./data/cocoa-metadata.tsv --p-formula stage --p-reference-levels "stage::early" --o-differentials ./data/kegg_reaction_differentials.qza

In [ ]:
mosh composition da-barplot --i-data ./data/kegg_reaction_differentials.qza --p-significance-threshold 0.05 --p-effect-size-threshold 1.1 --o-visualization ./data/kegg_reaction_differentials.qzv

You can view the plot by downloading the .qzv file and opening it using http://view.qiime2.org. To download the file click on the folder symbol to the left, open the `materials` folder, then the `data` folder, and choose download from the dot menu next to the filename. Note that the visualization that opens in your browser window has multiple tabs, allowing you to also view the citation and data provenance information associated with this output.

We can look up these reactions in the KEGG database to view detailed information about each reaction. E.g., here is the reaction that is most enriched in "late" stage cocoa fermentations:
https://www.genome.jp/dbget-bin/www_bget?rn:R10850

To view other reactions, simply adjust the reaction number (the last part of the URL)

### 1.3.1 Supervised learning-based enrichment prediction
Alternatively, we could train a machine-learning classifier to predict group membership based on pathway abundance. The features identified as most predictive/important in this model will be more enriched/depleted in the different metadata categories.

Usually machine-learning models should be trained with a large number of samples for a reliable model. We are using a tiny dataset and will obtain an unreliable model, but will do so anyway here for the purposes of demonstration:

In [ ]:
mosh sample-classifier classify-samples-ncv --i-table ./data/kegg_reaction_ft.qza --m-metadata-file ./data/cocoa-metadata.tsv --m-metadata-column stage --p-random-state 9 --o-predictions ./data/predictions.qza --o-feature-importance ./data/feature-importance.qza --o-probabilities ./data/probabilities.qza

In [ ]:
mosh sample-classifier confusion-matrix --i-predictions ./data/predictions.qza --i-probabilities ./data/probabilities.qza --m-truth-file ./data/cocoa-metadata.tsv --m-truth-column stage --o-visualization ./data/confusion-matrix.qzv

Inspect the `confusion-matrix.qzv` visualization to view the model accuracy.

To see which pathways are enriched in each group, we can generate a heatmap to visualize the abundance of each pathway in each sample, labeled by fermentation stage. We will visualize the top 50 most important features (i.e., the 50 that contribute most to the predictive performance of the model):

In [ ]:
mosh sample-classifier heatmap --i-table ./data/kegg_reaction_ft.qza --i-importance ./data/feature-importance.qza --m-sample-metadata-file ./data/cocoa-metadata.tsv --m-sample-metadata-column stage --p-feature-count 50 --o-heatmap ./data/rf-heatmap.qzv --o-filtered-table ./data/filtered_ft.qza

## 1.4 Pathway Visualization
Clearly the (relative) abundances of certain pathways change significantly between early and late fermentation. What is changing and why?

To help contextualize this information, you may want to [inspect the taxonomic composition](https://view.qiime2.org/visualization/?src=https://raw.githubusercontent.com/bokulich-lab/moshpit-docs/main/moshpit_docs/data/mags-derep-bar-plot-90.qzv) of these samples ([see here](https://moshpit.readthedocs.io/en/latest/chapters/03_taxonomic_classification/mags.html) for details on how this was generated).

The `annotate` plugin contains an action which can plot the KEGG terms from the ANCOM-BC result onto the complete metabolic pathway map. The command below will generate maps where red colors will indicate pathways enriched in the late (or middle) stage of the fermentation process, as compared to the early stage. Similarly, the blue color will show you the depleted pathways. You can click on the "Open on iPATH3" button to open the map in the interactive [iPATH3 viewer](https://pathways.embl.de/ipath3.cgi) to explore further. 

In [ ]:
mosh annotate map-pathways --i-differentials ./data/kegg_reaction_differentials.qza --p-significance-threshold 0.05 --o-visualization ./data/kegg_reaction_mapped.qzv

# 2. Functional diversity analysis

Now that we have pathway counts per sample, we can use this information to compare samples and test whether pathway abundances are associated with any specific metadata information.

## 2.1. Beta diversity
We will start by calculating a [Bray-curtis dissimilarity matrix](https://en.wikipedia.org/wiki/Bray%E2%80%93Curtis_dissimilarity) to measure the dissimilarity between each sample, based on observed frequency of different KEGG pathways in each sample.

In [ ]:
mosh diversity beta --i-table ./data/kegg_reaction_ft.qza --p-metric braycurtis --o-distance-matrix ./data/braycurtis_dist.qza

Next, we will perform [principal coordinates analysis (PCoA)](https://en.wikipedia.org/wiki/Multidimensional_scaling) from the obtained Bray-curtis matrix.

In [ ]:
mosh diversity pcoa --i-distance-matrix ./data/braycurtis_dist.qza --o-pcoa ./data/braycurtis_pcoa.qza

Visualization time! Let’s plot the PCoA results.

In [ ]:
mosh emperor plot --i-pcoa ./data/braycurtis_pcoa.qza --m-metadata-file ./data/cocoa-metadata.tsv --o-visualization ./data/kegg-pcoa.qzv

To test for significant differences in functional profile, we can run a [PERMANOVA](https://en.wikipedia.org/wiki/Permutational_analysis_of_variance) test. We will test the effect of fermentation stage and seed type on the functional profiles.

In [ ]:
mosh diversity adonis --i-distance-matrix ./data/braycurtis_dist.qza --m-metadata-file ./data/cocoa-metadata.tsv --p-formula 'stage+seed' --o-visualization ./data/kegg-adonis.qzv

## 2.2. Alpha diversity
We can also calculate alpha diversity, using [Shannon entropy](https://en.wikipedia.org/wiki/Diversity_index#Shannon_index) to assess whether the diversity of KEGG orthologs observed in each sample increases or decreases over time, or whether their are differences between seed type.

In [ ]:
mosh diversity alpha --i-table ./data/kegg_reaction_ft.qza --p-metric shannon --o-alpha-diversity ./data/shannon_vector.qza

We can visualize this a few different ways, e.g., as a line plot or boxplot:

In [ ]:
mosh vizard lineplot --m-metadata-file ./data/cocoa-metadata.tsv --m-metadata-file ./data/shannon_vector.qza --p-x-measure timepoint --p-y-measure shannon_entropy --p-group-by seed --o-visualization ./data/shannon_lineplot.qzv

In [ ]:
mosh vizard boxplot --m-metadata-file ./data/cocoa-metadata.tsv --m-metadata-file ./data/shannon_vector.qza --p-distribution-measure shannon_entropy --p-group-by stage --o-visualization ./data/shannon_boxplot.qzv

...and test for statistical significance, e.g., with an ANOVA test to test for effects of seed type and timepoint on entropy:

In [ ]:
mosh longitudinal anova --m-metadata-file ./data/cocoa-metadata.tsv --m-metadata-file ./data/shannon_vector.qza --p-formula 'shannon_entropy~timepoint+seed' --o-visualization ./data/shannon_anova.qzv

# 3. Time to Explore

If you made it this far, feel free to explore further! Several plugins are already installed in the computational environment that we are using (this is done in the setup step above). These plugins introduce different functionality. You can use the `--help` flag to read the documentation for any given plugin, as well as for the individual actions in each plugin. Give it a try! Explore a bit and see if there are any other interesting methods that you could apply to this dataset.

_hint: check out other actions in the `longitudinal` and `diversity` plugins!_

In [ ]:
mosh --help